<a href="https://colab.research.google.com/github/himanshu-nakrani/TMLC-Gen-ai-projects/blob/main/RAG_Retrieval_Optimization_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-cohere langchain pdfminer.six chromadb rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.6 MB/s eta 0:00:

In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["COHERE_API_KEY"] = user_secrets.get_secret("COHERE_KEY")
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Hybrid Search

In [3]:
# Define the directory where the Chroma database will persist data
persist_directory = "/kaggle/working/"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)


<ipython-input-3-f6fcfcfaf463>:7: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embedding = CohereEmbeddings(


In [4]:
def load_data_to_vectordb(file_path,source):
  global bm25_retriever
  # Loop through a list of PDF files to process
  pages = []

  for pdf_name in [file_path]:
      # Open each PDF file in binary mode
      with open(pdf_name, 'rb') as f:
          # Extract text from the PDF using the extract_text_pdf_miner function
          text = extract_text_pdf_miner(f)

          # Clean the extracted text by removing newline characters and joining into a single string
          cleaned_text = " ".join(text.split("\n"))

          # Initialize a list to store document chunks
          docs = []

          # Create a text splitter to divide the text into manageable chunks
          # Each chunk has a maximum size of 2048 characters with a 512-character overlap
          splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

          # Split the cleaned text into chunks and wrap each chunk in a Document object
          for chunk in splitter.split_text(cleaned_text):
              docs.append(Document(page_content=chunk, metadata={"retrived_from":source,"source": pdf_name}))
              pages.append(Document(page_content=chunk, metadata={"retrived_from":source,"source": pdf_name}))
      # Create a Chroma collection from the processed documents
      # Use the specified persist directory and embedding model for storage and retrieval
      if source == 1:
        bm25_retriever = BM25Retriever.from_documents(pages)
      else:
        db = Chroma.from_documents(
            documents=docs,
            persist_directory=persist_directory,
            embedding=embedding
        )

load_data_to_vectordb(file_path="/kaggle/input/100-ml-interview-questions-and-answers/100-Machine-Learning-Interview-Questions-and-Answers.pdf",source=1)
# load_data_to_vectordb(file_path="/content/1506.02640v5.pdf",source=2)

In [5]:
# Initialize the BM25 retriever
bm25_retriever.k = 2  # Retrieve top 2 results
print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [6]:
# Initialize retriever
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever_chromadb = docsearch.as_retriever(search_kwargs={"k": 5})

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever_chromadb], weights=[0.3, 0.7]
)

<ipython-input-6-641eb3961709>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [7]:
# Example query
query = "What is k in KNN algorithm?"

# Retrieve relevant documents/products
docs = ensemble_retriever.invoke(query)

docs

[Document(metadata={'retrived_from': 1, 'source': '/kaggle/input/100-ml-interview-questions-and-answers/100-Machine-Learning-Interview-Questions-and-Answers.pdf'}, page_content='same to the model that was trained on the remaining data set to get the required predictions.  2. K-Fold Cross-Validation: Here, the data is divided into k subsets so that every time, one  among the k subsets can be used as a validation set, and the other k-1 subsets are used as the training set  3. Stratified K-Fold Cross-Validation: It works on imbalanced data. 4. Leave-P-Out Cross-Validation: Here, we leave p data points out of the training data out of the n data points, then we use the n-p samples to train the model and p points for the validation set.  5. Differences Between The Bagging And Boosting Algorithms?  \x0cBagging  Boosting  It is a method that merges the same type of predictions.  It is a method that merges the different types of predictions.  It decreases the variance, not the bias  It decrease

In [8]:
#Extract and print only the page content from each document
import pandas as pd

retrieval_df = pd.DataFrame()

page_content = []
retrieval_source = []
pdf_source = []

for doc in docs:
    page_content.append(doc.page_content)
    retrieval_source.append(doc.metadata['retrived_from'])
    pdf_source.append(doc.metadata['source'])

retrieval_df['page_content'] = page_content
retrieval_df['retrieval_source'] = retrieval_source
retrieval_df['pdf_source'] = pdf_source

retrieval_df.head(10)

,page_content,retrieval_source,pdf_source
0,same to the model that was trained on the rema...,1,/kaggle/input/100-ml-interview-questions-and-a...
1,100 Machine Learning Interview Questions and A...,1,/kaggle/input/100-ml-interview-questions-and-a...


## Re Ranking

In [9]:
# Define the directory where the Chroma database will persist data
persist_directory = "/kaggle/working/"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)

In [10]:
def load_data_to_vectordb(file_path,source):
  # Loop through a list of PDF files to process

  for pdf_name in [file_path]:
      # Open each PDF file in binary mode
      with open(pdf_name, 'rb') as f:
          # Extract text from the PDF using the extract_text_pdf_miner function
          text = extract_text_pdf_miner(f)

          # Clean the extracted text by removing newline characters and joining into a single string
          cleaned_text = " ".join(text.split("\n"))

          # Initialize a list to store document chunks
          docs = []

          # Create a text splitter to divide the text into manageable chunks
          # Each chunk has a maximum size of 2048 characters with a 512-character overlap
          splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

          # Split the cleaned text into chunks and wrap each chunk in a Document object
          for chunk in splitter.split_text(cleaned_text):
              docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))
      # Create a Chroma collection from the processed documents
      # Use the specified persist directory and embedding model for storage and retrieval

      db = Chroma.from_documents(
            documents=docs,
            persist_directory=persist_directory,
            embedding=embedding
        )

load_data_to_vectordb(file_path="/kaggle/input/100-ml-interview-questions-and-answers/100-Machine-Learning-Interview-Questions-and-Answers.pdf",source=1)
# load_data_to_vectordb(file_path="/content/1506.02640v5.pdf",source=2)

In [11]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [12]:
docsearch = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [13]:
# Initialize an empty DataFrame with specified columns
non_rerank_df = pd.DataFrame(columns=['Text', 'source', 'relevance_score'])

# Perform similarity search using a preconfigured document search tool
# This retrieves the top 3 documents based on relevance to the query
res_docs = docsearch.similarity_search_with_relevance_scores("What is k in KNN algorithm?", k=3)

# Loop through the retrieved documents and populate the DataFrame
for doc in res_docs:
    non_rerank_df = non_rerank_df._append(
        {
            'Text': doc[0].page_content,  # Extract the page content (text) from the document
            'source': doc[0].metadata['source'],  # Extract the source metadata
            'relevance_score': doc[1]  # Extract the relevance score
        },
        ignore_index=True  # Ensure the new row is appended without overwriting existing rows
    )

# Display the first 3 rows of the DataFrame
non_rerank_df.head(3)

<ipython-input-13-15e4bf0c64e3>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  non_rerank_df = non_rerank_df._append(


,Text,source,relevance_score
0,used to examine the association of independent...,/kaggle/input/100-ml-interview-questions-and-a...,0.257310
1,same to the model that was trained on the rema...,/kaggle/input/100-ml-interview-questions-and-a...,0.170640
2,kernel 7. Bessel function of the first kind Ke...,/kaggle/input/100-ml-interview-questions-and-a...,0.148103


In [14]:
# Import and initialize the reranker for document compression
compressor = CohereRerank()
# CohereRerank is a model or tool designed to rerank documents based on their relevance.

# Create a ContextualCompressionRetriever for improved document retrieval
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,               # Use the reranker as the base compression mechanism
    base_retriever=docsearch.as_retriever()   # Use the existing document search tool as the base retriever
)
# The ContextualCompressionRetriever combines the base retriever's results with reranking
# to provide more contextually relevant and concise results.


<ipython-input-14-f983c412f7af>:2: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank()


In [15]:
# Initialize an empty DataFrame with specified columns
source_df = pd.DataFrame(columns=['Text', 'source', 'relevance_score'])

# Retrieve compressed documents relevant to the query using the contextual compression retriever
compressed_docs = compression_retriever.get_relevant_documents("What is k in KNN algorithm?")

# Loop through the first 3 compressed documents and populate the DataFrame
for i in range(3):
    source_df = source_df._append(
        {
            'Text': compressed_docs[i].page_content,  # Extract the content of the document
            'source': compressed_docs[i].metadata['source'],  # Extract the source information
            'relevance_score': compressed_docs[i].metadata['relevance_score']  # Extract the relevance score
        },
        ignore_index=True  # Ensure the new row is appended without overwriting existing rows
    )

# Display the first 3 rows of the DataFrame
source_df.head(3)

<ipython-input-15-ed8f874a587e>:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents("What is k in KNN algorithm?")
<ipython-input-15-ed8f874a587e>:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  source_df = source_df._append(


,Text,source,relevance_score
0,used to examine the association of independent...,/kaggle/input/100-ml-interview-questions-and-a...,0.922622
1,kernel 7. Bessel function of the first kind Ke...,/kaggle/input/100-ml-interview-questions-and-a...,0.784319
2,same to the model that was trained on the rema...,/kaggle/input/100-ml-interview-questions-and-a...,0.528001


We can see the last result is different and the relevance scores are also different but they are according to the relevance model